# HiERO Step Localization (Clean Version)

**Function**: Use the HiERO model to perform step segmentation on EgoVLP features.
**Features**:
1. Fixed environment dependency conflicts (Hydra/NetworkX).
2. Used advanced Spectral Clustering algorithm.
3. Automatically handled PyTorch `weights_only` security error.

In [ ]:
# 1. Environment Setup
import os

# Clone HiERO repository
if not os.path.exists('/content/HiERO'):
    !git clone https://github.com/T-Larm/HiERO_for_egovlp.git /content/HiERO

# Install core dependencies (using verified stable versions)
print("📦 Installing dependencies...")
!pip install -q hydra-core omegaconf einops torch_kmeans networkx

# Install PyTorch Geometric (auto-match current Colab Torch version)
!pip install -q torch-geometric
!pip install -q torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.5.1+cu124.html

print("✅ Environment installation complete! Please proceed to the next step.")

In [ ]:
# 2. Mount Drive and Path Configuration
import sys
from google.colab import drive

# Mount Google Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive', force_remount=True)

# Add HiERO to system path
if '/content/HiERO' not in sys.path:
    sys.path.insert(0, '/content/HiERO')

# ================= Path Configuration =================
# 1. Annotation file path (If in Drive, modify here; otherwise, auto-clone project)
PROJECT_ROOT = "/content/aml-2025-mistake-detection-gp"
if not os.path.exists(PROJECT_ROOT):
    !git clone https://github.com/T-Larm/aml-2025-mistake-detection-gp.git $PROJECT_ROOT

ANNOTATIONS_PATH = os.path.join(PROJECT_ROOT, "annotations/annotation_json/complete_step_annotations.json")

# 2. Feature and model paths (Based on your description)
EGOVLP_FEATURES_DIR = "/content/drive/MyDrive/AMLproject/our_features/gopro/segments/egovlp"
HIERO_CHECKPOINT = "/content/drive/MyDrive/AMLproject/Captain_Cook_dataset/hiero_egovlp/hiero_egovlp.pth"
OUTPUT_DIR = "/content/drive/MyDrive/AMLproject/extension1_outputs"

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"📂 Reading features: {EGOVLP_FEATURES_DIR}")
print(f"📂 Output results: {OUTPUT_DIR}")

In [ ]:
# 3. Load HiERO Model (Fixed Version)
import torch
import yaml
import hydra
from models.hiero import HiERO

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if os.path.exists(HIERO_CHECKPOINT):
    print("🚀 Loading model...")
    
    # Read Checkpoint (Disable weights_only security check to prevent errors)
    try:
        checkpoint = torch.load(HIERO_CHECKPOINT, map_location='cpu', weights_only=False)
    except TypeError:
        checkpoint = torch.load(HIERO_CHECKPOINT, map_location='cpu')
    
    # Process configuration
    if 'config' in checkpoint:
        config = checkpoint['config']
    else:
        config = {'model': {'conv': 'TDGC', 'hidden_size': 256, 'k': 2.0, 'n_layers': 2}}

    model_config = config.get('model', {}).copy()
    
    # Clean up Hydra residual parameters
    for key in ['_target_', '_recursive_', '_convert_']:
        model_config.pop(key, None)
    
    if isinstance(model_config.get('conv'), str):
        model_config['conv'] = {'name': model_config['conv']}

    # Initialize
    hiero_model = HiERO(input_size=256, **model_config)
    
    # Load weights
    state_dict = checkpoint.get('state_dict', checkpoint.get('model', checkpoint))
    new_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
    hiero_model.load_state_dict(new_state_dict, strict=False)
    
    hiero_model = hiero_model.to(device).eval()
    print("✅ HiERO model loaded successfully!")
else:
    print(f"❌ Model file not found: {HIERO_CHECKPOINT}")

In [ ]:
# 4. Define core segmentation function (includes Spectral Clustering and post-processing)
from sklearn.cluster import SpectralClustering
from torch_geometric.data import Data
import numpy as np

def detect_steps_with_hiero(features, model, device, n_clusters='auto', use_spectral=True, stride_seconds=None, fps=30):
    """
    Args:
        stride_seconds: Feature extraction stride in seconds (e.g., 1.0 for 1s). If None, uses stride=16 frames.
        fps: Video frame rate (default: 30)
    """
    T, D = features.shape

    # 1. Auto-estimate number of clusters
    if n_clusters == 'auto':
        n_clusters = max(3, min(T // 30, 15))
        n_clusters = int(n_clusters)

    # 2. Construct graph data (with all required attributes for HiERO)
    x = torch.from_numpy(features).float().to(device)
    edge_index = []
    for i in range(T - 1):
        edge_index.append([i, i + 1])
        edge_index.append([i + 1, i])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous().to(device)
    
    # Calculate stride_frames based on metadata or default
    if stride_seconds is not None:
        # Use stride from file metadata (e.g., 1s_1s.npz means 1 second stride)
        stride_frames = stride_seconds * fps
    else:
        # Default EgoVLP config: stride=16 frames
        stride_frames = 16
    
    # Create Data object with all attributes HiERO expects (following official implementation)
    graph_data = Data(x=x, edge_index=edge_index)
    graph_data.batch = torch.zeros(T, dtype=torch.long, device=device)
    graph_data.pos = ((0.5 + torch.arange(T, dtype=torch.float, device=device)) * stride_frames / fps).unsqueeze(1)  # Real timestamps in seconds
    graph_data.indices = torch.arange(T, dtype=torch.long, device=device)  # Frame indices
    graph_data.mask = torch.ones(T, dtype=torch.bool, device=device)  # Valid frame mask

    # 3. Model inference
    with torch.no_grad():
        try:
            output = model(graph_data)
            if isinstance(output, Data):
                hiero_features = output.x
            elif isinstance(output, dict):
                hiero_features = output.get('x', output.get('features', x))
            else:
                hiero_features = output
            
            if len(hiero_features) > T: hiero_features = hiero_features[:T]
            hiero_features = hiero_features.cpu().numpy()
        except Exception as e:
            print(f"  ⚠️ HiERO inference failed, using original features: {e}")
            hiero_features = features

    # 4. Spectral Clustering
    features_norm = hiero_features / (np.linalg.norm(hiero_features, axis=1, keepdims=True) + 1e-8)
    affinity = features_norm @ features_norm.T
    clustering = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', random_state=42, assign_labels='kmeans')
    labels = clustering.fit_predict(affinity)

    # 5. Convert to boundaries (with post-processing)
    boundaries = []
    current_start = 0
    for i in range(1, len(labels)):
        if labels[i] != labels[current_start]:
            if i - current_start >= 5: # Minimum length filtering
                boundaries.append((current_start, i - 1))
                current_start = i
    if len(labels) - current_start >= 5:
        boundaries.append((current_start, len(labels) - 1))
    
    # Extract Embedding
    step_embeddings = []
    for s, e in boundaries:
        step_embeddings.append(hiero_features[s:e+1].mean(axis=0))
    
    return boundaries, np.stack(step_embeddings) if step_embeddings else np.zeros((0, 256))

print("✅ Core algorithm function defined")

In [ ]:
# 5. Batch Process Videos (Main Loop)
import json
import re
from tqdm import tqdm

# Prepare ID list
with open(ANNOTATIONS_PATH, 'r') as f:
    annotations = json.load(f)

available_features = {f.split('_')[0] + '_' + f.split('_')[1] for f in os.listdir(EGOVLP_FEATURES_DIR) if f.endswith('.npz')}
common_ids = list(set(annotations.keys()).intersection(available_features))

print(f"🚀 Starting processing {len(common_ids)} videos...")

hiero_results = {}
failed = []

for vid in tqdm(common_ids):
    try:
        # Load features
        f_name = [f for f in os.listdir(EGOVLP_FEATURES_DIR) if f.startswith(vid)][0]
        data = np.load(os.path.join(EGOVLP_FEATURES_DIR, f_name))
        feats = data['arr_0'] if 'arr_0' in data else data['features']
        
        # Clean NaNs
        if np.isnan(feats).any(): feats = np.nan_to_num(feats)
        
        # Parse stride from filename (e.g., "xxx_1s_1s.npz" means stride=1s)
        stride_match = re.search(r'_(\d+\.?\d*)s_', f_name)
        stride_seconds = float(stride_match.group(1)) if stride_match else None
        
        # Run HiERO
        bounds, embs = detect_steps_with_hiero(feats, hiero_model, device, stride_seconds=stride_seconds, fps=30)
        
        # Record results
        anno = annotations[vid]
        has_error = any(s.get('has_errors', False) for s in anno.get('steps', []))
        
        hiero_results[vid] = {
            'boundaries': bounds,
            'step_embeddings': embs,
            'video_label': 1 if has_error else 0,
            'activity': anno.get('activity_name', 'unknown')
        }
    except Exception as e:
        failed.append(vid)

print(f"\n✅ Processing complete! Success: {len(hiero_results)}, Failed: {len(failed)}")

# Save results
save_path = os.path.join(OUTPUT_DIR, 'hiero_step_embeddings.npz')
np.savez(save_path, results=hiero_results)
print(f"💾 Results saved to: {save_path}")